<a href="https://colab.research.google.com/github/KimHee-Sun/class2022Spring/blob/main/huggingface_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
import gradio as gr

- huggingface의 모델을 그대로 가져와서 바로 실행할 수도 있음 (gpt2)

### [Image classification](https://huggingface.co/tasks/image-classification)

e.g. https://huggingface.co/google/vit-base-patch16-224 \
How to use


☝ **정확히 만들 수는 없더라도, 어떤 것을 의미하는 지는 알아야 함**

In [ ]:
!pip install transformers

In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests
#1) 필요한 것을 import 해오기

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
#2) 필요한 이미지를 가져와야 함 (업로드하지 않을거면 url로 가져오기, jpg를 통해 이미지인 것을 알 수 있음)
image = Image.open(requests.get(url, stream=True).raw)
#3) 이 이미지 안에는 데이터가 숫자로 있다는 것을 알아야 함

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
#pretrained: 행렬, 데이터로부터 잘 만들어진 행렬 (돈의 가치가 있는 행렬)
#5) feature_extractor: extraction을 할 수 있는 모델 -> 현재 3) 단계에서 숫자 정보가 있는데 (칼라면 3array, 흑백이면 2array), 이것을 다 쓰기에는 너무나 많으므로 압축적으로 쓸 수 있는 정보를 뽑는 것
#6) Classification: extracted 된 feature 정보를 가지고 classify하는 것

inputs = feature_extractor(images=image, return_tensors="pt")
#이러한 feature data (5)를 inputs으로 
outputs = model(**inputs)
#model의 입력으로 들어가는 것, output이 classify된 정보가 나오는 것
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])
#해당 그림을 Egyptian cat이라고 인식한 것, 1,000개의 idx 즉 class가 있는데 그것 중에 하나를 predict 해내는 것임
#1,000개의 class에 대한 확률이 모두 나오고, 이 중에 가장 확률이 높은 것 (argmx)를 찾았는데 이것이 Egyptian Cat이라고 보면됨 -> 1,000개의 숫자를 다 합치면 1!
#현재 id2lable은 script의 형태, 숫자가 쭉 나오는 것

Demo in Gradio

In [4]:
def func (image):
  #위에 있는 것의 일부를 떼와서 function화
  #image가 input ( model.config.id2label[predicted_class_idx])) -> 여기 부분!
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  # model predicts one of the 1000 ImageNet classes
  predicted_class_idx = logits.argmax(-1).item()
  predicted_class = model.config.id2label[predicted_class_idx]
  #출력, 변수로 받은 것 
  #image를 받아서 그것의 class를 text로 return
  return predicted_class
  # model.config.id2label[predicted_class_idx]) 여기 부분인 것!

In [ ]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/tiger.jpg"
os.system("curl " + url + " > tiger.jpg")
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/dog.jpg"
os.system("curl " + url + " > dog.jpg")

In [ ]:
gr.Interface(fn=func, inputs='image', outputs='text', examples = ['tiger.jpg', 'dog.jpg']).launch()
#function, inputs, outputs 정해주고 출력유형을 정해서 (image, text) launch
#examples로 사진 두개를 넣기 위해서 url로 이미지 사진 두개 받아줌 (반드시 list로 넣어주어야 함) -> demo를 만드는 gradio에서 사용할 수 있는 강점임 

### [Fill-Mask](https://huggingface.co/tasks/fill-mask)

e.g. https://huggingface.co/bert-base-uncased \
How to use

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
#이 model을 가져와서 unmasker라고 하겠다
unmasker("Hello I'm a [MASK] model.")
#이 model을 실행

Demo in Gradio

In [8]:
import pandas as pd
def func (text):
  unmasker = pipeline('fill-mask', model='bert-base-uncased')
  result = unmasker(text)
  df = pd.DataFrame(result)
  #위 모델의 결과값이 list 형태, 따라서 dataframe으로 만들 수 있음
  return df

In [13]:
examples = ["Hello I'm a [MASK] model.", "It is raining outside. I feel [MASK]."]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

### [Token classification](https://huggingface.co/tasks/token-classification)

e.g. https://huggingface.co/dslim/bert-base-NER \
How to use

- 정보 중에서 더욱 salient한 것을 recognize해내는 것 = NER!(LOC, ORG, PER, MISC)

In [ ]:
!pip install transformers

recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
#image classification과 유사하게 pretained를 불러옴 -> tokenizer라는 모델로 받았음
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
#각 단어를 classification 해주는 것을 NER, 4가지 종류

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
#위에 있는 example 문장을 가지고 위에 nlp 모델을 돌림
print(ner_results)
#이것도 list이므로 df할 수 있음

Demo in Gradio

In [17]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
def func (text):
  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
  nlp = pipeline("ner", model=model, tokenizer=tokenizer)
  result = nlp(text)
  df = pd.DataFrame(result)
  #list로 dictionary 안에 여러개가 있을 때 바로 dataframe화해줄 수 있음
  return df

In [ ]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

### [Sentence similarity](https://huggingface.co/tasks/sentence-similarity)

e.g. https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 \
How to use

- 모든 text, sound, image의 raw data는 숫자로 되어 있다고 하더라도, 바로 model input으로 넣어줄 수 없음 -> 따라서 feature로 만들기도 하고, vectors, embeddings라고 하기도 함

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "it is one example writing"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
#이 model의 input으로 위의 문장을 쓰면 embeddings로 출력해준다
#embeddings란? [[~~ 384개의 숫자] [~~384개의 숫자]]가 있음 -> 이것을 embeddings for text 1, text2라고 부르며 feature, vector (숫자들의 열이니까)
#-> 이것을 가지고 유사도를 어떻게 찾는가? goodnotes 참고 (**매우 매우 중요**)
#=similarity between two vectors (=embeddings) = 두 각도 값의 cos vector 값 =cosine similarity => 차원이 많아진다고 하더라도, 두 벡터는 원점과 함께 평면을 이루고 삼각형을 만들게 되어 있음
print(embeddings)

In [ ]:
len(embeddings[1])

In [ ]:
cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
#cosine similarity 계산 해주는 것, 두개의 vector (=embeddings)를 넣어주면 됨 (각 text의 384개의 숫자)
cosine_scores

Demo in Gradio

In [21]:
def func (text1, text2):
  from sentence_transformers import SentenceTransformer, util
  model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  embeddings = model.encode([text1, text2])
  cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
  return cosine_scores

In [22]:
examples = [["This is an example sentence", "it is one example writing"], ["A frog is hopping near the pond", "I love Korean Food"]]

In [ ]:
gr.Interface(fn=func, inputs=['text', 'text'], outputs='number', examples = examples).launch()